In [1]:
import json
import os
import sys
import traceback
from typing import Dict, Any

# Constants - adjust these to match your original values
cost_per_kwh_local = 12.26  # Adjust to your local electricity cost
to_usd_conversion = 0.018   # Adjust to your local currency to USD conversion rate

def recalculate_efficiency_metrics(metrics: Dict[str, Any]) -> Dict[str, Any]:
    """
    Recalculates cost efficiecny metrics using the correct currency conversion formula
    
    :param metrics: Single log entry dictionary
    :return: Updated metrics dictionary with corrected efficiency values
    """
    # Use val_accuracy instead of accuracy
    val_accuracy = metrics.get("val_accuracy", 0)
    training_time_hours = metrics.get("training_time_hours", 0)
    
    # Calculate total power (assuming you have both CPU and GPU power)
    cpu_power = metrics.get("cpu_power", 0)
    gpu_power = metrics.get("gpu_power", 0)
    total_power = cpu_power + (gpu_power if gpu_power not in [None, "N/A"] else 0)
    
    # Calculate energy and costs
    energy_kwh = (total_power / 1000) * training_time_hours
    cost_local = energy_kwh * cost_per_kwh_local
    cost_usd = cost_local * to_usd_conversion
    
    # Initialize with N/A values
    metrics['cost_efficiency_local'] = "N/A"
    metrics['cost_per_accuracy_point_local'] = "N/A"
    metrics['cost_efficiency_usd'] = "N/A"
    metrics['cost_per_accuracy_point_usd'] = "N/A"
    metrics['power_efficiency'] = "N/A"
    metrics['time_efficiency'] = "N/A"
    
    # Calculate cost efficiency metrics
    if cost_local > 0:
        # Accuracy % per local currency
        metrics["cost_efficiency_local"] = (val_accuracy * 100) / cost_local
        # Local currency per 1% accuracy
        metrics['cost_per_accuracy_point_local'] = cost_local / (val_accuracy * 100)

    if cost_usd > 0:
        # Accuracy % per dollar
        metrics["cost_efficiency_usd"] = (val_accuracy * 100) / cost_usd
        # Dollar per 1% accuracy
        metrics['cost_per_accuracy_point_usd'] = cost_usd / (val_accuracy * 100)

    if total_power > 0:
        # Accuracy % per watt
        metrics['power_efficiency'] = (val_accuracy * 100) / total_power

    if training_time_hours > 0:
        # Accuracy % per hour
        metrics['time_efficiency'] = (val_accuracy * 100) / training_time_hours
    
    return metrics

def reprocess_log_file(file_path: str, backup: bool = True) -> None:
    """
    Reprocesses a log file to use val_accuracy instead of accuracy for efficiency metrics.
    
    :param file_path: Path to the log file
    :param backup: Whether to create a backup of the original file
    """
    if not os.path.exists(file_path):
        print(f"Error: File {file_path} does not exist")
        return
    
    # Create backup if requested
    if backup:
        backup_path = file_path + ".backup"
        if not os.path.exists(backup_path):
            print(f"Creating backup: {backup_path}")
            with open(file_path, 'r') as original, open(backup_path, 'w') as backup_file:
                backup_file.write(original.read())
        else:
            print(f"Backup already exists: {backup_path}")
    
    print(f"Processing log file: {file_path}")
    
    try:
        # Read the log file
        with open(file_path, 'r') as f:
            content = f.read().strip()
        
        # Handle empty file
        if not content:
            print("Warning: File is empty")
            return
        
        # Split into individual JSON objects (one per line)
        log_lines = content.split('\n')
        processed_entries = []
        
        for i, line in enumerate(log_lines):
            if line.strip():  # Skip empty lines
                try:
                    # Parse JSON
                    entry = json.loads(line)
                    
                    # Recalculate efficiency metrics
                    updated_entry = recalculate_efficiency_metrics(entry)
                    processed_entries.append(updated_entry)
                    
                except json.JSONDecodeError as e:
                    print(f"Error parsing line {i+1}: {e}")
                    print(f"Problematic line: {line}")
                    continue
        
        # Write back to file
        with open(file_path, 'w') as f:
            for entry in processed_entries:
                f.write(json.dumps(entry) + '\n')
        
        print(f"Successfully processed {len(processed_entries)} entries")
        print("Efficiency metrics have been recalculated using val_accuracy")
        
    except Exception as e:
        exc_type, exc_obj, exc_tb = sys.exc_info()
        fname = os.path.split(exc_tb.tb_frame.f_code.co_filename)[1]
        print(f"Error processing file: {e} at {fname}:{exc_tb.tb_lineno}")
        print("Traceback:")
        print(traceback.format_exc())

def reprocess_multiple_files(file_paths: list, backup: bool = True) -> None:
    """
    Reprocesses multiple log files.
    
    :param file_paths: List of file paths to process
    :param backup: Whether to create backups of original files
    """
    for file_path in file_paths:
        reprocess_log_file(file_path, backup)
        print("-" * 50)

In [2]:
output_root_dir: str = "./../out/"

logs: list = []

In [3]:
for dirs in os.scandir(output_root_dir):
    if dirs.is_dir():
        dir_name = dirs.name
        dir = os.path.join(output_root_dir, dir_name)

        for root, dirs, files in os.walk(dir):
            for file in files:
                if file.endswith(".log"):
                    logs.append(os.path.join(root, file))

print(f"Found {len(logs)} log files to process")
logs

Found 24 log files to process


['./../out/cpu\\training_metrics_Intel_i5-11400H-EarlyStopping-ReduceLROnPlateau-run_1.log',
 './../out/cpu\\training_metrics_Intel_i5-11400H-EarlyStopping-run_1.log',
 './../out/cpu\\training_metrics_Intel_i5-11400H-ReduceLROnPlateau-run_1.log',
 './../out/cpu\\training_metrics_Intel_i5-11400H-run_1.log',
 './../out/gpu\\training_metrics-RTX_3050_Ti_Laptop-EarlyStopping-ReduceLROnPlateau-run_1.log',
 './../out/gpu\\training_metrics-RTX_3050_Ti_Laptop-EarlyStopping-ReduceLROnPlateau-run_2.log',
 './../out/gpu\\training_metrics-RTX_3050_Ti_Laptop-EarlyStopping-ReduceLROnPlateau-run_3.log',
 './../out/gpu\\training_metrics-RTX_3050_Ti_Laptop-EarlyStopping-ReduceLROnPlateau-run_4.log',
 './../out/gpu\\training_metrics-RTX_3050_Ti_Laptop-EarlyStopping-ReduceLROnPlateau-run_5.log',
 './../out/gpu\\training_metrics-RTX_3050_Ti_Laptop-EarlyStopping-run_1.log',
 './../out/gpu\\training_metrics-RTX_3050_Ti_Laptop-EarlyStopping-run_2.log',
 './../out/gpu\\training_metrics-RTX_3050_Ti_Laptop-Earl

In [4]:
reprocess_multiple_files(logs, backup = False)

Processing log file: ./../out/cpu\training_metrics_Intel_i5-11400H-EarlyStopping-ReduceLROnPlateau-run_1.log
Successfully processed 78 entries
Efficiency metrics have been recalculated using val_accuracy
--------------------------------------------------
Processing log file: ./../out/cpu\training_metrics_Intel_i5-11400H-EarlyStopping-run_1.log
Successfully processed 45 entries
Efficiency metrics have been recalculated using val_accuracy
--------------------------------------------------
Processing log file: ./../out/cpu\training_metrics_Intel_i5-11400H-ReduceLROnPlateau-run_1.log


Successfully processed 100 entries
Efficiency metrics have been recalculated using val_accuracy
--------------------------------------------------
Processing log file: ./../out/cpu\training_metrics_Intel_i5-11400H-run_1.log
Successfully processed 100 entries
Efficiency metrics have been recalculated using val_accuracy
--------------------------------------------------
Processing log file: ./../out/gpu\training_metrics-RTX_3050_Ti_Laptop-EarlyStopping-ReduceLROnPlateau-run_1.log
Successfully processed 100 entries
Efficiency metrics have been recalculated using val_accuracy
--------------------------------------------------
Processing log file: ./../out/gpu\training_metrics-RTX_3050_Ti_Laptop-EarlyStopping-ReduceLROnPlateau-run_2.log
Successfully processed 70 entries
Efficiency metrics have been recalculated using val_accuracy
--------------------------------------------------
Processing log file: ./../out/gpu\training_metrics-RTX_3050_Ti_Laptop-EarlyStopping-ReduceLROnPlateau-run_3.log
